In [5]:
from __future__ import annotations

from pathlib import Path
import torch
from IPython.display import Audio

from f5_tts.api import F5TTS  # Python API exists and is used in examples :contentReference[oaicite:2]{index=2}


In [ ]:
DATA_DIR = Path("/home/SpeakerRec/BioVoice/data/") 
TTS_DIR = DATA_DIR / "tts/f5"


REF_AUDIO = DATA_DIR / "wavs/Yoav_001.wav"
REF_TEXT = "My voice is my password."

GEN_TEXT = "Hello world, I'm Yoav."
OUT_WAV = TTS_DIR / "yoav_001_cloned.wav"
USE_GPU = torch.cuda.is_available()
DEVICE = "cuda" if USE_GPU else "cpu"


In [7]:
print("CUDA available:", USE_GPU)

try:
    f5 = F5TTS(device=DEVICE)
except TypeError:
    f5 = F5TTS()

print("F5TTS loaded.")



CUDA available: True
Download Vocos from huggingface charactr/vocos-mel-24khz

vocab :  /home/SpeakerRec/BioVoice/venv/lib/python3.10/site-packages/f5_tts/infer/examples/vocab.txt
token :  custom
model :  /home/SpeakerRec/.cache/huggingface/hub/models--SWivid--F5-TTS/snapshots/84e5a410d9cead4de2f847e7c9369a6440bdfaca/F5TTS_v1_Base/model_1250000.safetensors 

F5TTS loaded.


In [8]:
assert REF_AUDIO.exists(), f"Missing REF_AUDIO: {REF_AUDIO}"
OUT_WAV.parent.mkdir(parents=True, exist_ok=True)
wav, sr, spec = f5.infer(
    ref_file=str(REF_AUDIO),
    ref_text=REF_TEXT,
    gen_text=GEN_TEXT,
    file_wave=str(OUT_WAV),
    # file_spec=str(OUT_WAV.with_suffix(".png")),  # optional spectrogram output
    remove_silence=False,  # you can toggle this if you want tighter samples
)

print("Wrote:", OUT_WAV.resolve(), "| sr:", sr)
Audio(str(OUT_WAV))


Converting audio...
Using cached preprocessed reference audio...
Using custom reference text...

ref_text   My voice is my password. 
gen_text 0 Hello world, I'm Idan.


Generating audio in 1 batches...


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.56s/it]

Wrote: /home/SpeakerRec/BioVoice/data/tts/f5/idan_001_cloned.wav | sr: 24000
